# Packaging Model

## About

- Package the given model using Torch Model Archive
- Write a custom handler to support pre processing and post processing

## Working directory

orignal model and the traced model we saved from before

In [1]:
!ls ../artifacts/model/

amazon


directory contains tokenizer/ vocab / pytorch model

In [2]:
!ls ../artifacts/model/amazon

config.json	    setup_config.json	     tokenizer_config.json
index_to_name.json  special_tokens_map.json  training_args.bin
pytorch_model.bin   tokenizer.json	     vocab.txt


In [3]:
!ls ../artifacts/model/amazon_trace

ls: cannot access '../artifacts/model/amazon_trace': No such file or directory


## Torch Model Archiver

TorchServe required the model and its dependant artifacts to be packaged in a single file. 

[torch-model-archiver](https://pypi.org/project/torch-model-archiver/) is a python package that can package the artifacts to a mar file

In [ ]:
%%bash 

torch-model-archiver --help

package the model artifact and actual handler code

In [ ]:
%%bash

cd ..
pwd

ARTIFACT_BASE_DIR="artifacts/model/amazon_trace"

MODEL_NAME="pt_classifier"
MODEL_VERSION="1.0"
MODEL_STORE="${ARTIFACT_BASE_DIR}/model_store"
MODEL_SERIALIZED_FILE="${ARTIFACT_BASE_DIR}/traced_model.pt"

TOKENIZER_FILES="${ARTIFACT_BASE_DIR}/tokenizer_config.json,${ARTIFACT_BASE_DIR}/special_tokens_map.json,${ARTIFACT_BASE_DIR}/vocab.txt,${ARTIFACT_BASE_DIR}/tokenizer.json"
MODEL_EXTRA_FILES="${ARTIFACT_BASE_DIR}/index_to_name.json,${ARTIFACT_BASE_DIR}/setup_config.json,${TOKENIZER_FILES}"




mkdir -p $MODEL_STORE

torch-model-archiver --model-name ${MODEL_NAME} \
--version ${MODEL_VERSION} \
--serialized-file ${MODEL_SERIALIZED_FILE} \
--export-path ${MODEL_STORE} \
--extra-files ${MODEL_EXTRA_FILES} \
--handler ./serving/handler.py \
--force



In [ ]:
!cat ../serving/handler.py

if you would live to serve through Docker, lets copy the `model_store` artifact relative to the DockerFile folder

In [ ]:
%%bash
cd .. 
cp artifacts/model/amazon_trace/model_store/* serving/model_store
cp artifacts/model/amazon_trace/setup_config.json serving/model_store/

## Torchserve

> TorchServe is a performant, flexible and easy to use tool for serving PyTorch eager mode and torschripted models.

Ref: [TorchServe Docs](https://pytorch.org/serve/)

below command starts torchserve

In [ ]:
%%bash
cd ..
torchserve --ts-config ./serving/config.properties \
--start --model-store ./serving/model_store --ncs



In [ ]:
!cat ../serving/config.properties 

below command stops torchserve

In [ ]:
#torchserve --stop

List all the models loaded

In [ ]:
!curl "http://localhost:9081/models"

get details on the model `pt_classifier`

In [ ]:
!curl http://localhost:9081/models/pt_classifier

sample prediction

In [ ]:
! curl -X POST http://localhost:9080/predictions/pt_classifier \
        -H 'Content-Type: application/json' \
        -d '{"text":"herbal tea","request_id":"test_id"}'

In [ ]:
sample prediction from a file

In [ ]:
%%bash
cd ..
curl -X POST http://localhost:9080/predictions/pt_classifier \
        -H 'Content-Type: application/json' \
        -d @serving/sample_input.json